In [7]:
import os
import pickle
import torch
from rdkit import Chem
import pandas as pd
import numpy as np
torch.set_default_tensor_type('torch.cuda.FloatTensor')
from AttentiveFP import save_smiles_dicts, get_smiles_array

In [8]:
model_details_df = pd.read_csv('../data/model_details.csv', index_col= 'index')
raw_filename = "../data/dataset.csv"

In [9]:
for best_name in model_details_df.index:

    batch_size = 128
    fingerprint_dim = model_details_df['fingerprint_dim']['{}'.format(best_name)]
    radius = model_details_df['radius']['{}'.format(best_name)]
    T = model_details_df['T']['{}'.format(best_name)]
    moddel_path = model_details_df['model_path']['{}'.format(best_name)]
    model = torch.load(model_details_df['model_path']['{}'.format(best_name)])
    output_units_num = 1

    feature_filename = raw_filename.replace('.csv','.pickle')
    filename = raw_filename.replace('.csv','')
    smiles_tasks_df = pd.read_csv(raw_filename)
    smilesList = smiles_tasks_df.smiles.values
    print("number of all smiles: ",len(smilesList))
    atom_num_dist = []
    remained_smiles = []
    canonical_smiles_list = []
    for smiles in smilesList:
        try:        
            mol = Chem.MolFromSmiles(smiles)
            atom_num_dist.append(len(mol.GetAtoms()))
            remained_smiles.append(smiles)
            canonical_smiles_list.append(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True))
        except:
            print(smiles)
            pass
    print("number of successfully processed smiles: ", len(remained_smiles))
    smiles_tasks_df = smiles_tasks_df[smiles_tasks_df["smiles"].isin(remained_smiles)]
    smiles_tasks_df['cano_smiles'] =canonical_smiles_list
    assert canonical_smiles_list[8]==Chem.MolToSmiles(Chem.MolFromSmiles(smiles_tasks_df['cano_smiles'][8]), isomericSmiles=True)
    smilesList = [smiles for smiles in canonical_smiles_list]

    if os.path.isfile(feature_filename):
        feature_dicts = pickle.load(open(feature_filename, "rb" ))
    else:
        feature_dicts = save_smiles_dicts(smilesList,filename)
    remained_df = smiles_tasks_df[smiles_tasks_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]

    model.eval()
    preList = np.arange(0,remained_df.shape[0])
    batch_list = []
    y_pred_label=[]
    for i in range(0, remained_df.shape[0], batch_size):
        batch = preList[i:i+batch_size]
        batch_list.append(batch) 
    for counter, pre_batch in enumerate(batch_list):
        batch_df = remained_df.loc[pre_batch,:]
        smiles_list = batch_df.cano_smiles.values
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)
        atoms_prediction, mol_prediction, _ = model(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask))
        
        mol_prediction_np = mol_prediction.cpu()
        y_pred_label.append(mol_prediction_np.detach().numpy())
    y_pred_label_list = [i[0] for item in y_pred_label for i in item]
    remained_df['y_pred_label'] = y_pred_label_list
    remained_df.to_csv("../results/labeled_{}.csv".format(best_name),mode = 'a',index =False)

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL AttentiveFP.AttentiveLayers.Fingerprint was not an allowed global by default. Please use `torch.serialization.add_safe_globals([AttentiveFP.AttentiveLayers.Fingerprint])` or the `torch.serialization.safe_globals([AttentiveFP.AttentiveLayers.Fingerprint])` context manager to allowlist this global if you trust this class/function.

Check the documen